<a href="https://colab.research.google.com/github/NickFury89/Car-race-game/blob/master/%D0%A1%D0%B5%D1%80%D0%B2%D0%B8%D1%81_%D0%BC%D0%BE%D0%BD%D0%B8%D1%82%D0%BE%D1%80%D0%B0_%D0%BA%D0%BD%D0%B8%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bеб-скрейпер на Python для сбора метаданных и текстов книг из онлайн-магазинов и библиотек:**

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_book(url):
    
    # Отправляем GET-запрос на страницу с книгой
    
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')

    # Извлекаем метаданные книги
    
    title = soup.find('h1', {'class': 'book-title'}).text.strip()
    
    author = soup.find('div', {'class': 'book-author'}).find('a').text.strip()
    
    genre = soup.find('span', {'class': 'genre'}).find('a').text.strip()
    
    description = soup.find('div', {'class': 'book-annotation'}).text.strip()
    
    cover_url = soup.find('img', {'class': 'book-image'})['src']

    # Извлекаем текст книги
    
    book_text = ''
    
    for page_num in range(1, 101):
        
        page_url = f'{url}/chapter-{page_num}'
        
        response = requests.get(page_url)
        
        page_soup = BeautifulSoup(response.text, 'html.parser')
        
        page_text = page_soup.find('div', {'class': 'reader-container'}).text.strip()
        
        book_text += page_text + '\n'

    # Возвращаем словарь с метаданными и текстом книги
    
    return {
        
        'title': title,
        
        'author': author,
        
        'genre': genre,
        
        'description': description,
        
        'cover_url': cover_url,
        
        'book_text': book_text,
    }


# Код обработки естественного языка для извлечения функций библиотеки из текста книг

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def extract_library_functions(book_text):
    
    # Разбиваем текст книги на предложения и токены
    
    sentences = sent_tokenize(book_text)
    
    tokens = word_tokenize(book_text.lower())

    # Определяем список стоп-слов и лемматизатор
    
    stop_words = set(stopwords.words('english'))
    
    lemmatizer = WordNetLemmatizer()

    # Извлекаем глаголы, прилагательные и существительные
    
    pos_tags = nltk.pos_tag(tokens)
    
    verbs = [lemmatizer.lemmatize(word) for word, tag in pos_tags if tag.startswith('VB')]
    
    adjectives = [lemmatizer.lemmatize(word) for word, tag in pos_tags if tag.startswith('JJ')]
    
    nouns = [lemmatizer.lemmatize(word) for word, tag in pos_tags if tag.startswith('NN')]

    # Отфильтровываем стоп-слова и возвращаем уникальные значения
    
    verbs = list(set(verbs) - stop_words)
    
    adjectives = list(set(adjectives) - stop_words)
    
    nouns = list(set(nouns) - stop_words)

    # Возвращаем словарь с глаголами, прилагательными и существительными
    
    return {
        
        'verbs': verbs,
        
        'adjectives': adjectives,
        
        'nouns': nouns,
    }


# Машинное обучение

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import joblib
import sqlite3

# Подключаемся к базе данных и создаем таблицу для результатов
conn = sqlite3.connect('books.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS model_results 
             (model_name text, accuracy real)''')

# Загружаем данные и разделяем их на обучающую и тестовую выборки
data = pd.read_csv('books_data.csv')
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['genre'], test_size=0.2, random_state=42)

# Обучаем модели на обучающей выборке
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': MultinomialNB(),
}
for name, model in models.items():
    model.fit(X_train, y_train)

    # Оцениваем точность моделей на тестовой выборке
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Сохраняем модели и результаты в базу данных
    joblib.dump(model, f'{name}.joblib')
    c.execute("INSERT INTO model_results VALUES (?, ?)", (name, accuracy))

# Сохраняем изменения в базе данных и закрываем соединение
conn.commit()
conn.close()


# База данных

In [ ]:
CREATE TABLE books (
    id INTEGER PRIMARY KEY,
    title TEXT,
    author TEXT,
    genre TEXT,
    language TEXT,
    publication_date TEXT,
    publisher TEXT,
    text TEXT,
    features TEXT,
    analysis_result TEXT
);


# Чтобы сохранить данные в базе данных, мы можем использовать трафик sqlite3в Python. Вот пример кода, который добавляет данные в таблицу books:

In [ ]:
import sqlite3

conn = sqlite3.connect('books.db')
cursor = conn.cursor()

# Добавляем данные в таблицу
cursor.execute("INSERT INTO books (title, author, genre, language, publication_date, publisher, text, features, analysis_result) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", (title, author, genre, language, publication_date, publisher, text, features, analysis_result))
conn.commit()

# Закрываем соединение с базой данных
conn.close()


Для создания внешнего вида для поиска книг по созданию параметров и страниц для просмотра информации о книгах и их оценках можно использовать веб-фреймворк, например Flask. Ниже представлен пример кода для создания простого веб-приложения с возможностью поиска книг по жанру и просмотра информации о книгах:


In [ ]:
from flask import Flask, render_template, request
import sqlite3

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/search', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        genre = request.form['genre']
        conn = sqlite3.connect('books.db')
        c = conn.cursor()
        c.execute("SELECT * FROM books WHERE genre=?", (genre,))
        books = c.fetchall()
        conn.close()
        return render_template('search_results.html', books=books)
    else:
        return render_template('search.html')

@app.route('/book/<int:book_id>')
def book(book_id):
    conn = sqlite3.connect('books.db')
    c = conn.cursor()
    c.execute("SELECT * FROM books WHERE id=?", (book_id,))
    book = c.fetchone()
    c.execute("SELECT * FROM ratings WHERE book_id=?", (book_id,))
    ratings = c.fetchall()
    conn.close()
    return render_template('book.html', book=book, ratings=ratings)

if __name__ == '__main__':
    app.run(debug=True)


Развернуть систему на облачном сервисе для обеспечения масштабируемости и отказоустойчивости. используйте облачные сервисы для хранения данных и файлы с шаблонами машинного обучения. Настроены высокие масштабы и мониторинг состояния системы.